<a href="https://colab.research.google.com/github/saykim/ds/blob/main/voicetotext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install google-cloud-speech

     |████████████████████████████████| 92kB 2.8MB/s 


In [ ]:
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.6.0/audio-0.6.0.tar.gz
!tar -xvzf audio-0.6.0.tar.gz
!ls -l ./audio/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   629  100   629    0     0   3418      0 --:--:-- --:--:-- --:--:--  3437
100  192k  100  192k    0     0   567k      0 --:--:-- --:--:-- --:--:--  567k
audio/
audio/2830-3980-0043.wav
audio/Attribution.txt
audio/4507-16021-0012.wav
audio/8455-210777-0068.wav
audio/License.txt
total 260
-rw-r--r-- 1 501 staff 63244 Nov 18  2017 2830-3980-0043.wav
-rw-r--r-- 1 501 staff 87564 Nov 18  2017 4507-16021-0012.wav
-rw-r--r-- 1 501 staff 82924 Nov 18  2017 8455-210777-0068.wav
-rw-r--r-- 1 501 staff   340 May 14  2018 Attribution.txt
-rw-r--r-- 1 501 staff 18652 May 12  2018 License.txt


In [ ]:
TESTCASES = [
  {
    'filename': 'audio/sol_20200528_1.wav',
    'text': 'experience proves this',
    'encoding': 'LINEAR16',
    'lang': 'ko-KR'}
]

In [ ]:
from typing import Tuple
import wave

def read_wav_file(filename) -> Tuple[bytes, int]:
    with wave.open(filename, 'rb') as w:
        rate = w.getframerate()
        frames = w.getnframes()
        buffer = w.readframes(frames)

    return buffer, rate

def simulate_stream(buffer: bytes, batch_size: int = 4096):
    buffer_len = len(buffer)
    offset = 0
    while offset < buffer_len:
        end_offset = offset + batch_size
        buf = buffer[offset:end_offset]
        yield buf
        offset = end_offset

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving eng-braid-278706-c9fff9cfddea.json to eng-braid-278706-c9fff9cfddea.json
User uploaded file "eng-braid-278706-c9fff9cfddea.json" with length 2349 bytes


In [ ]:
!pwd
!ls -l ./eng-braid-278706-c9fff9cfddea.json

/content
-rw-r--r-- 1 root root 2349 Jun  5 01:22 ./eng-braid-278706-c9fff9cfddea.json


In [ ]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './eng-braid-278706-c9fff9cfddea.json'

!ls -l $GOOGLE_APPLICATION_CREDENTIALS

-rw-r--r-- 1 root root 2349 Jun  5 01:22 ./eng-braid-278706-c9fff9cfddea.json


In [ ]:
from google.cloud import speech_v1
from google.cloud.speech_v1 import enums

def google_batch_stt(filename: str, lang: str, encoding: str) -> str:
    buffer, rate = read_wav_file(filename)
    client = speech_v1.SpeechClient()

    config = {
        'language_code': lang,
        'sample_rate_hertz': rate,
        'encoding': enums.RecognitionConfig.AudioEncoding[encoding]
    }

    audio = {
        'content': buffer
    }

    response = client.recognize(config, audio)
    # For bigger audio file, the previous line can be replaced with following:
    # operation = client.long_running_recognize(config, audio)
    # response = operation.result()

    for result in response.results:
        # First alternative is the most probable result
        alternative = result.alternatives[0]
        return alternative.transcript

# Run tests
for t in TESTCASES:
    print('\naudio file="{0}"    expected text="{1}"'.format(t['filename'], t['text']))
    print('google-cloud-batch-stt: "{}"'.format(
        google_batch_stt(t['filename'], t['lang'], t['encoding'])
    ))


audio file="audio/sol_20200528_1.wav"    expected text="experience proves this"


InvalidArgument: ignored

In [ ]:
from google.cloud import speech_v1
from google.cloud.speech_v1 import enums

def google_batch_stt(filename: str, lang: str, encoding: str) -> str:
    buffer, rate = read_wav_file(filename)
    client = speech_v1.SpeechClient()

    config = {
        'language_code': lang,
        'sample_rate_hertz': rate,
        'encoding': enums.RecognitionConfig.AudioEncoding[encoding]
    }

    audio = {
        'content': buffer
    }

    response = client.recognize(config, audio)
    # For bigger audio file, the previous line can be replaced with following:
    # operation = client.long_running_recognize(config, audio)
    # response = operation.result()

    for result in response.results:
        # First alternative is the most probable result
        alternative = result.alternatives[0]
        return alternative.transcript

# Run tests
for t in TESTCASES:
    print('\naudio file="{0}"    expected text="{1}"'.format(t['filename'], t['text']))
    print('google-cloud-batch-stt: "{}"'.format(
        google_batch_stt(t['filename'], t['lang'], t['encoding'])
    ))


audio file="audio/sol_20200528_1.wav"    expected text="experience proves this"


InvalidArgument: ignored

In [ ]:
!gcloud init

Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
compute:
  gce_metadata_read_timeout_sec: '0'

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  1

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

You must log in to continue. Would you like to log in (Y/n)?  y

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuser

In [ ]:
def transcribe_gcs(gcs_uri):
    from google.cloud import speech
    from google.cloud.speech import enums
    from google.cloud.speech import types
    client = speech.SpeechClient()
    audio = types.RecognitionAudio(uri=gcs_uri)
    config = types.RecognitionConfig(encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16, sample_rate_hertz=44100,language_code='ko-KR')
    operation = client.long_running_recognize(config, audio)
    response = operation.result()
    return response
response = transcribe_gcs("gs://529speechtotext/sol_0604_01.wav")
with open('/content/604script_file_1.txt', "w") as script:
    for result in response.results:
        script.write(u'{}'.format(result.alternatives[0].transcript)+"\n")
print("completed")


InvalidArgument: ignored

In [ ]:
from google.cloud import speech
client = speech.SpeechClient()
results = client.recognize(
    audio=speech.types.RecognitionAudio(
        uri='gs://529speechtotext/sol_0604_01.wav',
    ),
    config=speech.types.RecognitionConfig(
        encoding='LINEAR16',
        language_code='ko-KR',
        sample_rate_hertz=44100,
    ),
)

InvalidArgument: ignored

In [ ]:
#내 방식대로 변경, 내가 녹음한 파일변환
def transcribe_gcs(gcs_uri):
    from google.cloud import speech
    from google.cloud.speech import enums
    from google.cloud.speech import types
    client = speech.SpeechClient()
    audio = types.RecognitionAudio(uri=gcs_uri)
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,sample_rate_hertz=44100,language_code='ko-KR')
    operation = client.long_running_recognize(config, audio)
    response = operation.result()
    return response
response = transcribe_gcs("gs://529speechtotext/sol_0602_2.wav")
with open('/content/604_script1.txt', "w") as script:
    for result in response.results:
        script.write(u'{}'.format(result.alternatives[0].transcript)+"\n")
print("completed")

InvalidArgument: ignored

In [ ]:
def transcribe_gcs(gcs_uri):
    from google.cloud import speech
    from google.cloud.speech import enums
    from google.cloud.speech import types
    client = speech.SpeechClient()
    audio = types.RecognitionAudio(uri=gcs_uri)
    config = types.RecognitionConfig(
      encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16, sample_rate_hertz=44100,language_code='ko-KR')
    operation = client.long_running_recognize(config, audio)
    response = operation.result()
    return response
response = transcribe_gcs("gs://529speechtotext/sol_20200528_1.wav")
print(response.results)

In [ ]:
response[0]

In [ ]:
import chardet

In [ ]:
response.results[0]

In [ ]:
response.results[0].alternatives[0].transcript

'했던 거'

In [ ]:
for i in range(84):
  a = response.results[i].alternatives[0].transcript
  print(a)

했던 거
여기 시장을 보면
처음에 회사에서 먹고 먼저 한 거고요 나머지는 언제 받을 수 있을까요
지도에서 하는 거를 이제 회사에서 제가 처리를 하는데
이제 금형을 맞추기 위해서 너는 물입니다 그 때 생물은 동물을 만드는데 그 다음에 나중에 통화하게 위안을 얻고 있습니다 근데 그때 이제 또 최적화에서
 엄마
 데이터가 지금 이제 비용으로만 작성되면 되기 때문에 힘들다 소리 나게 되면 그때 그 정보를 올리는 인터넷으로 해 가지고
 지금은 제일 처음에 처음에 그 물 속에 있는 동물의 양을 알 수 없기 때문에 저희가 할 수 있는 최대 중화제를 나왔다가 그것보다 좀 곱게 처음부터 시작되는 있고 그다음에 운영하면서 진행되는 그
 하는게 맞을 것 같은데요 요거 하나 갖고 이게 어떤 거야 계속 그래서 히바리 되긴 하는데 탱크가 안에 들어 있기 때문에 공기 순환이 잘 되는 환경이면 근데 저희가 배에다가 되는데 어떤 데는 환기가 잘 돼 가지고 모자 그냥 그대로
 네 그걸로 얼마나 좋냐
 뭐를
 넘기시는 거를 환경이라고 했지만 이런 요소들에 따라서 배가 뭐 그런 것도 시간이 영양
 내가 맨 처음에 할 때부터 시작해서 실제로 뭘 해야지
 격하게 맞출 수 없지만요
 모든 모든 대해서 들어갈 수 있도록 사람들이 이제 설치를 하면서 그때 이제 대충 보니까 뭐 하고 있는데 그 일본이랑 씨방이 타이트하게 안 된다고 생각하기 때문에 그거를 약간 미세조정을 그거를 이제 사람이 하려고 하면 1년 정도 같이 배를 타고 가면서 한번 둘러보면서 조금 수정하고 강원도 수량은 이런 식으로 해야 될 거라고 생각하는데 그거를 이제 이런 기능을 탑재 하게 되면 끝날 스스로 좀 더 그때 막 이런 생각이 들거든요
 6시까지는 안 될 거 같고요
 루피 바람이 불어오는 거 보면은 8시간 아마도 안 본 거 같은데
 결정이 되는군
 현재 온도
 원래는 항상 아틀란티스로 줄어들 거라고 생각했는데 줄어들긴 하지만 저희가 생각한 것보다 약간 그냥 위치에서 일정표 담아 놓고 이제 출발이야 얼마나 되나 그런 걸 가지고 있었는데 